In [1]:
import pickle
import os
import json
from copy import deepcopy
import pandas as pd
import csv
from datetime import datetime, timedelta
from collections import Counter
import numpy as np

In [2]:
path = '/Users/fan/Desktop/Fullhouse/WCM/HospitalManagement/OrderAnalysis-master/'

In [3]:
fh = open(path + "2_EDDC_HF_2012_2018_ouput_0305.json", 'r')
data3 = json.load(fh)

In [4]:
len(data3)

2291

In [5]:
data = deepcopy(data3)

add eligibility

In [6]:
eligible=[]
c=0
c2=0
for pid in data3:
#     print(pid)
#     print(data3[pid]['age'])
    if len(data3[pid]['appt'])>1 and data3[pid]['service']=='MED':
        for date in sorted(data3[pid]['appt'],reverse=True):
            age=int(date[0:4])-data3[pid]['age']
            if age>18 and age<90:
                eligible.append(pid)
            elif data3[pid]['age']<0:
                c2+=1 #age limit  
            elif age>=90:
                c=c+1 #no age info
            break
print(len(eligible),c2,c)  ## 55 people don't have age, 400 people age >=90 or <=18

1651 55 400


In [7]:
for pid in data3:
    if pid in eligible:
        pass
    else:
        del data[pid]

see some dates below have 0 locations

In [19]:
locationpid={}
for pid in data:
    locationpid[pid]=[]
    for date in data3[pid]['appt']:
        if len(data3[pid]['appt'][date]['location'])>0:
            locationpid[pid].append(str(list(set(data3[pid]['appt'][date]['location']))).split('[')[1].split('-')[0].split(']')[0])
for pid in locationpid:
    locationpid[pid]=list(set(locationpid[pid]))

In [20]:
clientguid=[]
for pid in data:
    if data[pid]['clientID'] not in clientguid and pid in eligible:
        clientguid.append(data[pid]['clientID'])
print(len(clientguid))

1356


compute outcome measures for 30, 60, 90 day readmissions

In [21]:
readmission={}
readmissionvisitguids30=[]
readmissionvisitguids60=[]
readmissionvisitguids90=[]
c=0
s=0
r=0
for guid in clientguid:
    readmission[guid]=[]
for guid in clientguid:
    for pid in data:
        if int(guid)==int(data[pid]['clientID']):
            readmission[guid].append([data[pid]['admittime'],data[pid]['dischargetime'],pid])
            r=r+1
print("num unique pt guids",len(readmission))  
print("num visits",len(data))
l=0
a=0
b=0
for guid in readmission:
    if len(readmission[guid])>1:
        l=l+1
for guid in readmission:
    for i in range(len(readmission[guid])-1):
        s=s+1
        if (datetime.strptime(readmission[guid][i+1][0],'%Y-%m-%d %H:%M:%S')-datetime.strptime(readmission[guid][i][1],'%Y-%m-%d %H:%M:%S')).days>=30:
            c=c+1
        else:
            readmissionvisitguids30.append(readmission[guid][i+1][2])
            
        if (datetime.strptime(readmission[guid][i+1][0],'%Y-%m-%d %H:%M:%S')-datetime.strptime(readmission[guid][i][1],'%Y-%m-%d %H:%M:%S')).days<60:
            readmissionvisitguids60.append(readmission[guid][i+1][2])
        if (datetime.strptime(readmission[guid][i+1][0],'%Y-%m-%d %H:%M:%S')-datetime.strptime(readmission[guid][i][1],'%Y-%m-%d %H:%M:%S')).days<90:
            readmissionvisitguids90.append(readmission[guid][i+1][2])
        
readmissionvisitguids30=list(set(readmissionvisitguids30))
print("num unique pt guids with more than 1 admission",l)
print("num readmissions",s)
print("num re-admissions that are NOT 30-day",c)
print("num re-admissions that are 30-day",len(readmissionvisitguids30))
print("num re-admissions that are 60-day",len(readmissionvisitguids60))
print("num re-admissions that are 90-day",len(readmissionvisitguids90))

num unique pt guids 1356
num visits 1651
num unique pt guids with more than 1 admission 219
num readmissions 295
num re-admissions that are NOT 30-day 243
num re-admissions that are 30-day 52
num re-admissions that are 60-day 86
num re-admissions that are 90-day 121


In [22]:
for pid in data:
    data[pid]['30day_readmit']=0
    data[pid]['60day_readmit']=0
    data[pid]['90day_readmit']=0
    data[pid]['LOS']=(datetime.strptime(data[pid]['dischargetime'],'%Y-%m-%d %H:%M:%S') - datetime.strptime(data[pid]['admittime'], '%Y-%m-%d %H:%M:%S')).days
    if pid in readmissionvisitguids30:
        data[pid]['30day_readmit']=1
    if pid in readmissionvisitguids60:
        data[pid]['60day_readmit']=1
    if pid in readmissionvisitguids90:
        data[pid]['90day_readmit']=1

get list of unique diagnoses

In [27]:
diaglist = list()
for pid in data:
    if len(data[pid]['appt']) != 0:
        for date in sorted(iter(data[pid]['appt'])):
            if 'withinappt' not in data[pid]['appt'][date] or len(data[pid]['appt'][date]['withinappt']) == 0:
                for i in data[pid]['appt'][date]['diag']:
                    diaglist.append(i)
                    # for key, value in i.items():
                    #     diaglist.append(value)
            elif len(data[pid]['appt'][date]['withinappt']) > 0:
                for time in sorted(iter(data[pid]['appt'][date]['withinappt'])):
                    for i in data[pid]['appt'][date]['withinappt'][time]['diag']:
                        for key, value in i.items():
                            diaglist.append(value)

In [28]:
c_diag=Counter(diaglist)
c_diag.most_common()

[(nan, 6439),
 ('I50.9', 2631),
 ('R06.02', 1285),
 ('N17.9', 812),
 ('I48.91', 752),
 ('I50.23', 652),
 ('I10', 566),
 ('I50.33', 462),
 ('R07.9', 427),
 ('R06.00', 424),
 ('J18.9', 417),
 ('I25.10', 362),
 ('E11.9', 337),
 ('D64.9', 317),
 ('R09.02', 230),
 ('I50.21', 220),
 ('I50.20', 213),
 ('I50.30', 205),
 ('I34.0', 191),
 ('R60.0', 186),
 ('N18.9', 181),
 ('N39.0', 178),
 ('R10.9', 177),
 ('R41.82', 176),
 ('R05', 169),
 ('J44.1', 166),
 ('E87.70', 165),
 ('J44.9', 159),
 ('J96.00', 158),
 ('A41.9', 151),
 ('R79.89', 144),
 ('I50.1', 142),
 ('E78.5', 138),
 ('I95.9', 136),
 ('E87.1', 136),
 ('N18.6', 135),
 ('W19.XXXA', 135),
 ('R50.9', 133),
 ('D72.829', 131),
 ('I21.4', 130),
 ('R57.0', 129),
 ('E87.5', 127),
 ('R53.1', 124),
 ('I47.2', 122),
 ('I50.22', 119),
 ('J90', 119),
 ('M79.89', 118),
 ('J96.01', 117),
 ('I50.43', 116),
 ('K92.2', 114),
 ('R60.9', 107),
 ('R78.81', 106),
 ('Z51.5', 106),
 ('I25.5', 106),
 ('I42.9', 98),
 ('I47.1', 98),
 ('R41.0', 97),
 ('I35.0', 96),
 

In [29]:
diaglist=list(set(diaglist))

In [30]:
#create VTE outcome
import math
c=Counter(diaglist)
num=0
for i in c:
    if isinstance(i, float)==False:
        if i[0:3]=='I82':
            num=num+c[i]
num

19

In [31]:
orderlist = []
for pid in data:
    if len(data[pid]['appt']) != 0:
        for date in sorted(iter(data[pid]['appt'])):
            if 'withinappt' not in data[pid]['appt'][date] or len(data[pid]['appt'][date]['withinappt']) == 0:
                for icd in range(len(data[pid]['appt'][date]['proc'])):
                    orderlist.append(str(data[pid]['appt'][date]['proc'][icd]))
            elif len(data[pid]['appt'][date]['withinappt']) > 0:
                for time in sorted(iter(data[pid]['appt'][date]['withinappt'])):
                    for p in data[pid]['appt'][date]['withinappt'][time]['proc']:
                        orderlist.append(p)

In [32]:
c_order=Counter(orderlist)
c_order.most_common()

[('OSAM Lab Order Set', 36228),
 ('OSCommonly Used Orders - AM Draw', 33499),
 ('Lab_order', 23712),
 ('Sequential Compression Sleeves: Apply to lower extremities while in bed',
  6481),
 ('OSCTS AM Labs Order Set', 4579),
 ('OSMED - ED Admitted Patient Order Set', 4479),
 ('OSED Order Set - A General Comprehensive Order Set', 2442),
 ('Indwelling Urinary Catheter (Foley)', 1937),
 ('OSHeparin  Drip Order Set (Adult)', 1715),
 ('OSWarfarin Order Set', 1699),
 ('OSUrine Lab Order Set', 1567),
 ('OSMED - ED Admitted Patient Order Set (LMH)', 1514),
 ('IV, Peripheral Access', 1203),
 ('OSED Order Set - General (LMH)', 1178),
 ('OSCommonly Used Orders - Now', 1132),
 ('OSED Order Set - A General Comprehensive Order Set (LMH)', 1038),
 ('ECG/12 Lead-POC ED', 896),
 ('OSED Order Set  - A General Comprehensive Order Set', 861),
 ('Restraints  -', 860),
 ('OSED Order Set -  Abdominal/Flank Pain', 850),
 ('OSED Order Set - General', 819),
 ('OSPM Labs Order Set', 737),
 ('OSED Order set - Dyspn

In [33]:
orderlist=list(set(orderlist))

In [34]:
druglist = []
for pid in data:
    if len(data[pid]['appt']) != 0:
        for date in sorted(iter(data[pid]['appt'])):
            if 'withinappt' not in data[pid]['appt'][date] or len(data[pid]['appt'][date]['withinappt']) == 0:
                for icd in range(len(data[pid]['appt'][date]['drug'])):
                    druglist.append(data[pid]['appt'][date]['drug'][icd])
                    # druglist.append(data[pid]['appt'][date]['drug'][icd]['name'])
            elif len(data[pid]['appt'][date]['withinappt']) > 0:
                for time in sorted(iter(data[pid]['appt'][date]['withinappt'])):
                    for p in data[pid]['appt'][date]['withinappt'][time]['drug']:
                        druglist.append(p['name'])

In [35]:
c_drug=Counter(druglist)
c_drug.most_common()

[('Furosemide Inj', 5873),
 ('Potassium Chloride Oral', 3344),
 ('Magnesium Sulfate Inj', 2848),
 ('Warfarin Sodium Oral', 2679),
 ('Heparin Drip', 2614),
 ('Magnesium Oxide Oral', 2613),
 ('Bumetanide Inj', 2284),
 ('Acetaminophen Tab', 1827),
 ('Insulin Aspart Prandial Scale Pre-Meal', 1571),
 ('Furosemide Oral', 1530),
 ('Metoprolol Tartrate Tab', 1376),
 ('Potassium Chloride Inj', 1375),
 ('Albuterol Inh Soln', 1358),
 ('Heparin Inj', 1333),
 ('Potassium Chloride Oral Liq', 1302),
 ('Insulin Aspart Bedtime Scale', 1298),
 ('Dextrose 50% Inj', 1190),
 ('Ipratropium Bromide Inh Soln', 1095),
 ('Carvedilol Tab', 1066),
 ('Docusate Sodium Cap', 1017),
 ('Insulin Glargine Inj (Lantus)', 1005),
 ('Senna Oral', 1002),
 ('HYDROmorphone Inj', 971),
 ('Bumetanide Oral', 968),
 ('Atorvastatin Tab', 961),
 ('Influenza Virus Vaccine Inj >3 Years Old', 898),
 ('Aspirin EC Tab', 861),
 ('Pneumococcal 23-Valent Vacc (Pneumovax)', 844),
 ('Metolazone Oral', 844),
 ('Polyethylene Glycol Oral', 811),

In [36]:
druglist=list(set(druglist))

In [37]:
len(diaglist)

1584

In [38]:
len(orderlist)

266

In [39]:
len(druglist)

1069

Skip: get diagnoses, orders, and drugs observed by 90%+ of the patients

In [40]:
N=len(data)
N

1651

In [112]:
diagNotusedbyall=[]
used={}
for item in diaglist:
    used[item]={}  
    for pid in data:
        stop=False
        used[item][pid]=0
        for date in data[pid]['appt']:
            for i in data[pid]['appt'][date]['diag']:
                if i==item:
                    used[item][pid]=1
                    stop=True
                    break
            if stop==True:
                break
    if sum(used[item].values())>N*0.3 : #used by more than 30% of patients
        diagNotusedbyall.append(item)       

In [113]:
diagNotusedbyall

['N17.9', 'I50.9', 'R06.02']

In [114]:
orderNotusedbyall=[]
used={}
for item in orderlist:
    used[item]={}  
    for pid in data:
        stop=False
        used[item][pid]=0
        for date in data[pid]['appt']:
            for i in data[pid]['appt'][date]['proc']:
                if i==item:
                    used[item][pid]=1
                    stop=True
                    break
            if stop==True:
                break
    if sum(used[item].values())>N*0.8 : #used by more than 80% of patients
        orderNotusedbyall.append(item)           

In [115]:
len(c_order)

202

In [116]:
commonorder=[]
for i in c_order.most_common(10):
    commonorder.append(i[0])
commonorder

['OSAM Lab Order Set',
 'OSCommonly Used Orders - AM Draw',
 'Lab_order',
 'OSCTS AM Labs Order Set',
 'OSMED - ED Admitted Patient Order Set',
 'OSED Order Set - A General Comprehensive Order Set',
 'OSHeparin  Drip Order Set (Adult)',
 'OSWarfarin Order Set',
 'OSUrine Lab Order Set',
 'OSMED - ED Admitted Patient Order Set (LMH)']

In [117]:
commondrug=[]
for i in c_drug.most_common(10):
    commondrug.append(i[0])
commondrug

['Furosemide Inj',
 'Potassium Chloride Oral',
 'Magnesium Sulfate Inj',
 'Warfarin Sodium Oral',
 'Magnesium Oxide Oral',
 'Bumetanide Inj',
 'Potassium Chloride Oral Liq',
 'Potassium Chloride Inj',
 'Influenza Virus Vaccine Inj >3 Years Old',
 'Pneumococcal 23-Valent Vacc (Pneumovax)']

In [118]:
drugNotusedbyall=[]
used={}
for item in druglist:
    used[item]={}  
    for pid in data:
        stop=False
        used[item][pid]=0
        for date in data[pid]['appt']:
            for i in data[pid]['appt'][date]['drug']:
                if i==item:
                    used[item][pid]=1
                    stop=True
                    break
            if stop==True:
                break
    if sum(used[item].values())>N*0.5 : #used by more than 50% of patients
        drugNotusedbyall.append(item)     

In [119]:
drugNotusedbyall

['Furosemide Inj',
 'Influenza Virus Vaccine Inj >3 Years Old',
 'Pneumococcal 23-Valent Vacc (Pneumovax)']

In [120]:
# drugNotusedbyall=[]
# for item in druglist:
#     used=0
#     for pid in data:
#         for date in data[pid]['appt']:
#             for i in data[pid]['appt'][date]['drug']:
#                 if i==item:
#                     used+=1
    
#     if used>500 :
#         drugNotusedbyall.append(item)       

In [121]:
for pid in data:
    for date in sorted(iter(data[pid]['appt'])):
        data[pid]['appt'][date]['type'] = 'I'

In [122]:
data2 = deepcopy(data)

re-labeling orders

In [123]:
for pid in data:
    for date in sorted(data[pid]['appt']):
        data2[pid]['appt'][date]['diag'] = []
        data2[pid]['appt'][date]['proc'] = []
        data2[pid]['appt'][date]['drugclass'] = []
        data2[pid]['appt'][date]['drug'] = []
        data2[pid]['appt'][date]['user']=[]
#         for icd in range(len(data[pid]['appt'][date]['diag'])):
#             if data[pid]['appt'][date]['diag'][icd] in diagNotusedbyall and data[pid]['appt'][date]['diag'][icd] not in data2[pid]['appt'][date]['diag']: #ADD ONLY DIAGNOSIS THAT WE WANT
#                 data2[pid]['appt'][date]['diag'].append(data[pid]['appt'][date]['diag'][icd])

        for icd in range(len(data[pid]['appt'][date]['proc'])):
            if data[pid]['appt'][date]['proc'][icd] in commonorder and data[pid]['appt'][date]['proc'][icd] not in data2[pid]['appt'][date]['proc']:
                if data[pid]['appt'][date]['proc'][icd]=='OSAM Lab Order Set' or data[pid]['appt'][date]['proc'][icd]=='OSCTS AM Labs Order Set':
                    data2[pid]['appt'][date]['proc'].append('OSCommonly Used Orders - AM Draw')
                elif data[pid]['appt'][date]['proc'][icd]=='Lab_order':
                    pass
                elif data[pid]['appt'][date]['proc'][icd][0:2]=='OS':
                    data2[pid]['appt'][date]['proc'].append(str(data[pid]['appt'][date]['proc'][icd]))
                else:
#                     data2[pid]['appt'][date]['proc'].append(str(data[pid]['appt'][date]['proc'][icd]))
                    pass
                data2[pid]['appt'][date]['proc']=list(set(data2[pid]['appt'][date]['proc']))
#             if data[pid]['appt'][date]['proc'][icd]== 'Clinical Pathway - Heart Failure Order Set':
#                 data2[pid]['appt'][date]['proc'].append(str(data[pid]['appt'][date]['proc'][icd]))
#             if data[pid]['appt'][date]['proc'][icd]==  'Clinical Pathway - Heart Failure Order Set (LMH)':
#                 data2[pid]['appt'][date]['proc'].append('Clinical Pathway - Heart Failure Order Set')
                
#             if data[pid]['appt'][date]['proc'][icd]==  'ED Order set - Dyspnea/CHF/COPD/Asthma':
#                 data2[pid]['appt'][date]['proc'].append(str(data[pid]['appt'][date]['proc'][icd]))
#             if data[pid]['appt'][date]['proc'][icd]==   'ED Order set - Dyspnea/CHF/COPD/Asthma (LMH)':
#                 data2[pid]['appt'][date]['proc'].append('ED Order set - Dyspnea/CHF/COPD/Asthma')

        for icd in range(len(data[pid]['appt'][date]['drug'])):
            if data[pid]['appt'][date]['drug'][icd] in drugNotusedbyall and data[pid]['appt'][date]['drug'][icd] not in data2[pid]['appt'][date]['drugclass']:
                if str(data[pid]['appt'][date]['drug'][icd])== 'Furosemide Inj' or str(data[pid]['appt'][date]['drug'][icd])== 'Furosemide Oral':
                    data2[pid]['appt'][date]['drugclass'].append('Furosemide')
                    
                elif str(data[pid]['appt'][date]['drug'][icd])==  'Insulin Aspart Prandial Scale Pre-Meal' or str(data[pid]['appt'][date]['drug'][icd])==  'Insulin Aspart Bedtime Scale':
                    data2[pid]['appt'][date]['drugclass'].append('Insulin Aspart')
                elif str(data[pid]['appt'][date]['drug'][icd])==  'Potassium Chloride Inj' or str(data[pid]['appt'][date]['drug'][icd])==  'Potassium Chloride Oral':
                    data2[pid]['appt'][date]['drugclass'].append('Potassium Chloride') 
                else:
                    data2[pid]['appt'][date]['drugclass'].append(data[pid]['appt'][date]['drug'][icd])
                    
                data2[pid]['appt'][date]['drugclass']=list(set(data2[pid]['appt'][date]['drugclass']))

        for u in data[pid]['appt'][date]['user']:
            if u in Q1 and 'Q1' not in data2[pid]['appt'][date]['user']:
                data2[pid]['appt'][date]['user'].append('Q1')
            if u in Q3 and 'Q3' not in data2[pid]['appt'][date]['user']:
                data2[pid]['appt'][date]['user'].append('Q3')

In [125]:
pidcount=[]
for pid in data2:
    for date in data2[pid]['appt']:
        if len(data2[pid]['appt'][date]['drugclass'])>0 or len(data2[pid]['appt'][date]['proc'])>0:
            pidcount.append(pid)
            
#             print(data2[pid]['appt'][date]['drugclass'],data2[pid]['appt'][date]['proc'])
pidcount=list(set(pidcount))
print(len(pidcount))

1647


In [127]:
c=0
for pid in data2:
    if data2[pid]['dischargedisposition'] =='PATIENT EXPIRED':
        c=c+1
print(c)

78


split data by cluster

In [41]:
cluster=pd.read_excel(path + 'EDDC_order_2012_2018_cluster_MED_task_opioid_1651.xlsx', sheet_name='Sheet1')
cluster.columns

Index(['ylclientvisitguid', 'clientvisitguid', 'heartfailure',
       'generalcardiology', 'pulmonary', 'infectious', 'nephrology', 'aicd',
       'endocrinology', 'EDDC_all',
       ...
       'Performed, Methadone', 'Not Performed, Morphine', 'Overdue, Morphine',
       'Pending, Morphine', 'Performed, Morphine', 'Not Performed, Oxycodone',
       'Overdue, Oxycodone', 'Pending, Oxycodone', 'Performed, Oxycodone',
       'Performed opioids'],
      dtype='object', length=121)

In [42]:
cluster['payor'].value_counts()

Medicare      1121
Medicaid       221
Commercial     184
Dual            74
Selfpay         51
Name: payor, dtype: int64

In [48]:
cluster[['EDDC_cv3_notcancelled_noV0']].values

array([[1],
       [1],
       [1],
       ...,
       [2],
       [1],
       [1]])

In [226]:
data_C1=deepcopy(data)
for pid in list(data_C1.keys()):
    if  int(pid) in cluster['ylclientvisitguid'].values:
        if cluster.loc[cluster['ylclientvisitguid']==int(pid)][['EDDC_cv3_notcancelled_noV0']].values[0]==1:
            pass
            #if cluster.loc[cluster['ylclientvisitguid']==int(pid)][['payor']].values[0] != 'Selfpay':
            #    pass
            #else:
            #    del data_C1[pid]
        else: 
            del data_C1[pid]
    else:
        del data_C1[pid]

In [227]:
data_C2=deepcopy(data)
for pid in list(data_C2.keys()):
    if  int(pid) in cluster['ylclientvisitguid'].values:
        if cluster.loc[cluster['ylclientvisitguid']==int(pid)][['EDDC_cv3_notcancelled_noV0']].values[0]==2:
            pass
            #if cluster.loc[cluster['ylclientvisitguid']==int(pid)][['payor']].values[0] != 'Selfpay':
            #    pass
            #else:
            #    del data_C2[pid]
        else: 
            del data_C2[pid]
    else:
        del data_C2[pid]

In [228]:
data_C3=deepcopy(data)
for pid in list(data_C3.keys()):
    if  int(pid) in cluster['ylclientvisitguid'].values:
        if cluster.loc[cluster['ylclientvisitguid']==int(pid)][['EDDC_cv3_notcancelled_noV0']].values[0]==3:
            pass
            #if cluster.loc[cluster['ylclientvisitguid']==int(pid)][['payor']].values[0] != 'Selfpay':
            #    pass
            #else:
            #    del data_C3[pid]
        else: 
            del data_C3[pid]
    else:
        del data_C3[pid]

In [229]:
len(data_C1)

1254

In [230]:
len(data_C2)

296

In [231]:
len(data_C3)

99

most common for diagnoses

In [244]:
len(data_C1)

1254

In [245]:
diaglist_C1=[]
for pid in data_C1:
            for date in data_C1[pid]['appt']:
                for icd in data_C1[pid]['appt'][date]['diag']:
                    diaglist_C1.append(icd)
c=Counter(diaglist_C1)
c_most_common = [item for item in c.most_common(11)]
c_most_common

[(nan, 4167),
 ('I50.9', 2058),
 ('R06.02', 1022),
 ('I48.91', 537),
 ('N17.9', 444),
 ('I50.23', 426),
 ('I10', 424),
 ('R07.9', 338),
 ('I50.33', 329),
 ('R06.00', 321),
 ('J18.9', 266)]

In [246]:
orderlist_C1=[]
for pid in data_C1:
            for date in data_C1[pid]['appt']:
                for icd in data_C1[pid]['appt'][date]['proc']:
                    orderlist_C1.append(icd)
c=Counter(orderlist_C1)
c_most_common = [item for item in c.most_common(10)]
c_most_common

[('OSCommonly Used Orders - AM Draw', 13490),
 ('OSAM Lab Order Set', 12098),
 ('Lab_order', 9814),
 ('OSMED - ED Admitted Patient Order Set', 3223),
 ('OSED Order Set - A General Comprehensive Order Set', 1734),
 ('Sequential Compression Sleeves: Apply to lower extremities while in bed',
  1669),
 ('OSMED - ED Admitted Patient Order Set (LMH)', 1280),
 ('IV, Peripheral Access', 955),
 ('OSED Order Set - General (LMH)', 951),
 ('OSED Order Set - A General Comprehensive Order Set (LMH)', 782)]

In [247]:
## check
orderlistC1=[]
for pid in data:
    if int(pid) in cluster['ylclientvisitguid'].values:
        if cluster.loc[cluster['ylclientvisitguid']==int(pid)][['EDDC_cv3_notcancelled_noV0']].values[0]==1:
            #if cluster.loc[cluster['ylclientvisitguid']==int(pid)][['payor']].values[0] != 'Selfpay':
                for date in data[pid]['appt']:
                    for icd in data[pid]['appt'][date]['proc']:
                        orderlistC1.append(icd)
c=Counter(orderlistC1)
c_most_common = [item for item in c.most_common(10)]
c_most_common

[('OSCommonly Used Orders - AM Draw', 13490),
 ('OSAM Lab Order Set', 12098),
 ('Lab_order', 9814),
 ('OSMED - ED Admitted Patient Order Set', 3223),
 ('OSED Order Set - A General Comprehensive Order Set', 1734),
 ('Sequential Compression Sleeves: Apply to lower extremities while in bed',
  1669),
 ('OSMED - ED Admitted Patient Order Set (LMH)', 1280),
 ('IV, Peripheral Access', 955),
 ('OSED Order Set - General (LMH)', 951),
 ('OSED Order Set - A General Comprehensive Order Set (LMH)', 782)]

In [248]:
druglist_C1=[]
for pid in data_C1:
            for date in data_C1[pid]['appt']:
                for icd in data_C1[pid]['appt'][date]['drug']:
                    druglist_C1.append(icd)
c=Counter(druglist_C1)
c_most_common = [item for item in c.most_common(10)]
c_most_common

[('Furosemide Inj', 3391),
 ('Potassium Chloride Oral', 1512),
 ('Magnesium Oxide Oral', 1086),
 ('Warfarin Sodium Oral', 1080),
 ('Acetaminophen Tab', 1054),
 ('Magnesium Sulfate Inj', 1048),
 ('Furosemide Oral', 1042),
 ('Albuterol Inh Soln', 857),
 ('Insulin Aspart Prandial Scale Pre-Meal', 843),
 ('Heparin Drip', 832)]

In [249]:
## check
druglistC1=[]
for pid in data:
    if  int(pid) in cluster['ylclientvisitguid'].values:
        if cluster.loc[cluster['ylclientvisitguid']==int(pid)][['EDDC_cv3_notcancelled_noV0']].values[0]==1:
            #if cluster.loc[cluster['ylclientvisitguid']==int(pid)][['payor']].values[0] != 'Selfpay':
                for date in data[pid]['appt']:
                    for icd in data[pid]['appt'][date]['drug']:
                        druglistC1.append(icd)
c=Counter(druglistC1)
c_most_common = [item for item in c.most_common(10)]
c_most_common

[('Furosemide Inj', 3391),
 ('Potassium Chloride Oral', 1512),
 ('Magnesium Oxide Oral', 1086),
 ('Warfarin Sodium Oral', 1080),
 ('Acetaminophen Tab', 1054),
 ('Magnesium Sulfate Inj', 1048),
 ('Furosemide Oral', 1042),
 ('Albuterol Inh Soln', 857),
 ('Insulin Aspart Prandial Scale Pre-Meal', 843),
 ('Heparin Drip', 832)]

In [250]:
len(data_C1)

1254

tf_idf for diagnoses

In [181]:
#generate number of diagnosis
diaglistpt={}
for pid in data_C3:
    diaglistpt[pid]=[]
    for date in data_C3[pid]['appt']:
        for icd in data_C3[pid]['appt'][date]['diag']:
            if icd not in diaglistpt[pid] and pd.isna(icd)==False:
                diaglistpt[pid].append(icd)
    #print(pid,len(diaglistpt[pid]))

In [182]:
diag_tf={}
for item in diaglist:
    diag_tf[item]={}
    for pid in data_C3:
        diag_tf[item][pid]=0
        for date in data_C3[pid]['appt']:
            for icd in data_C3[pid]['appt'][date]['diag']:
                if icd==item:
                    diag_tf[item][pid]=1/len(diaglistpt[pid])

In [183]:
diag_df={}
for item in diaglist:
    diag_df[item]=[]
    for pid in data_C3:
        for date in data_C3[pid]['appt']:
            for icd in data_C3[pid]['appt'][date]['diag']:
                if icd==item and pid not in diag_df[item] and pd.isna(icd)==False:
                    diag_df[item].append(pid)

In [184]:
N=len(data_C3)
diag_idf={}
for item in diaglist:
    for pid in data_C3:
        diag_idf[item]=np.log(N/(len(diag_df[item])+1)) 

In [185]:
diag_td_idf={}
for item in diaglist:
    diag_td_idf[item]={}
    for pid in data_C3:
        diag_td_idf[item][pid]=diag_tf[item][pid]*diag_idf[item]

In [186]:
diag_td_idf_list={}
for item in diag_td_idf:
    diag_td_idf_list[item]=sum(diag_td_idf[item].values())

In [187]:
{k: v for k, v in sorted(diag_td_idf_list.items(), key=lambda item: item[1], reverse=True)}

{'I50.23': 1.7924042601589283,
 'I50.33': 1.7232563253281987,
 'D64.9': 1.6186836927261095,
 'I25.10': 1.617428936407004,
 'I48.91': 1.6033756335306923,
 'R06.02': 1.5918958573284632,
 'J18.9': 1.448205752686575,
 'J90': 1.4470660777222368,
 'R57.0': 1.3926557008941975,
 'E87.1': 1.392002287863133,
 'E87.5': 1.3745194641593415,
 'I10': 1.3564029233294668,
 'Z51.5': 1.3005256377621854,
 'Z95.811': 1.2824757798829303,
 'N18.9': 1.2781259448533726,
 'I25.5': 1.2253967244169954,
 'A41.9': 1.206291886442818,
 'R60.0': 1.2012516652268481,
 'R07.9': 1.1993704810770276,
 'I34.0': 1.1986184447717523,
 'R50.9': 1.1946610723597821,
 'R06.00': 1.1922731402385487,
 'I50.9': 1.1842723582312955,
 'N18.6': 1.1814084768970923,
 'I35.0': 1.172898123461075,
 'D72.829': 1.1724001892960527,
 'I95.9': 1.1621935159074823,
 'R41.82': 1.153452239803999,
 'I47.2': 1.15311555651441,
 'E87.70': 1.147199904195928,
 'Z95.810': 1.1452953605500884,
 'I27.2': 1.1374144511113147,
 'J96.00': 1.1339937232275599,
 'J96.01

tf_idf for orders

In [188]:
#generate number of orders
orderlistpt={}
for pid in data_C3:
    orderlistpt[pid]=[]
    for date in data_C3[pid]['appt']:
        for icd in data_C3[pid]['appt'][date]['proc']:
            if pd.isna(icd)==False:
                orderlistpt[pid].append(icd)
    #print(pid,len(diaglistpt[pid]))

In [189]:
order_tf={}
for item in orderlist:
    order_tf[item]={}
    for pid in data_C3:
        order_tf[item][pid]=0
        for date in data_C3[pid]['appt']:
            for icd in data_C3[pid]['appt'][date]['proc']:
                if icd==item and pd.isna(icd)==False:
                    order_tf[item][pid]=(order_tf[item][pid]+1)/len(orderlistpt[pid])

In [190]:
order_df={}
for item in orderlist:
    order_df[item]=[]
    for pid in data_C3:
        for date in data_C3[pid]['appt']:
            for icd in data_C3[pid]['appt'][date]['proc']:
                if icd==item and pid not in order_df[item] and pd.isna(icd)==False:
                    order_df[item].append(pid)

In [191]:
N=len(data_C3)
order_idf={}
for item in orderlist:
    for pid in data_C3:
        order_idf[item]=np.log(N/(len(order_df[item])+1)) 

In [192]:
order_td_idf={}
for item in orderlist:
    order_td_idf[item]={}
    for pid in data_C3:
        order_td_idf[item][pid]=order_tf[item][pid]*order_idf[item]

In [193]:
order_td_idf_list={}
for item in order_td_idf:
    order_td_idf_list[item]=sum(order_td_idf[item].values())

In [194]:
{k: v for k, v in sorted(order_td_idf_list.items(), key=lambda item: item[1], reverse=True)}

{'Cardiac monitor': 0.1448007859517943,
 'General Nursing': 0.12825950081721238,
 'ECG/12 Lead-POC ED': 0.1257867546394229,
 'B/p, hr, and temp': 0.12232824800301949,
 'OSAM Lab Order Set': 0.12128123797728148,
 'OSMED - ED Admitted Patient Order Set': 0.11280741812012281,
 'POC EPOC': 0.1106137740719538,
 'OSED Order Set - A General Comprehensive Order Set': 0.11056849871048609,
 'PICC Line Care': 0.10669295011614743,
 'Central Venous Line Care': 0.10524345490446764,
 'OSAnemia Workup Lab Order Set': 0.10134821905691116,
 'OSPM Labs Order Set': 0.10008669866571078,
 'IV, Peripheral Access': 0.09936187302510345,
 'OSCommonly Used Orders - AM Draw': 0.09607903099564857,
 'Indwelling Urinary Catheter Care': 0.0944195065079797,
 'Midline Catheter Care Order': 0.09268971668857563,
 'ECG/12 Lead-POC': 0.09217145264066406,
 'OSBlood Culture x2': 0.09182389860083899,
 'IV, Peripheral Access, Adult': 0.08984465871045298,
 'Juice for Hypoglycemic Treatment Only': 0.08757669112436368,
 'OSWarfar

tf_idf for drugs

In [195]:
druglistpt={}
for pid in data_C3:
    druglistpt[pid]=[]
    for date in data_C3[pid]['appt']:
        for icd in data_C3[pid]['appt'][date]['drug']:
            if pd.isna(icd)==False:
                druglistpt[pid].append(icd)
    #print(pid,len(diaglistpt[pid]))

In [196]:
drug_tf={}
for item in druglist:
    drug_tf[item]={}
    for pid in data_C3:
        drug_tf[item][pid]=0
        for date in data_C3[pid]['appt']:
            for icd in data_C3[pid]['appt'][date]['drug']:
                if icd==item and pd.isna(icd)==False:
                    drug_tf[item][pid]=(drug_tf[item][pid]+1)/len(druglistpt[pid])

In [197]:
drug_df={}
for item in druglist:
    drug_df[item]=[]
    for pid in data_C3:
        for date in data_C3[pid]['appt']:
            for icd in data_C3[pid]['appt'][date]['drug']:
                if icd==item and pid not in drug_df[item] and pd.isna(icd)==False:
                    drug_df[item].append(pid)

In [198]:
N=len(data_C3)
drug_idf={}
for item in druglist:
    for pid in data_C3:
        drug_idf[item]=np.log(N/(len(drug_df[item])+1))

In [199]:
drug_td_idf={}
for item in druglist:
    drug_td_idf[item]={}
    for pid in data_C3:
        drug_td_idf[item][pid]=drug_tf[item][pid]*drug_idf[item]

In [200]:
drug_td_idf_list={}
for item in drug_td_idf:
    drug_td_idf_list[item]=sum(drug_td_idf[item].values())

In [201]:
{k: v for k, v in sorted(drug_td_idf_list.items(), key=lambda item: item[1], reverse=True)}

{'Furosemide Oral': 0.20690618025941176,
 'Ferrous Sulfate Oral': 0.19933710545956854,
 'Metoprolol Tartrate Tab': 0.19311737170897197,
 'Apixaban Tab': 0.19293920036739778,
 'Clopidogrel Bisulfate Oral': 0.19001790183877318,
 'Tamsulosin Oral': 0.18926970280672117,
 'hydrALAZINE HCl Oral': 0.18688618578338964,
 'Pneumococcal 23-Valent Vacc (Pneumovax)': 0.18616290508683953,
 'Metoprolol XL Sustained Release Oral': 0.1861097990682626,
 'Influenza Virus Vaccine Inj >3 Years Old': 0.1858233458371417,
 'cefTRIAXone IV Inj': 0.1818547478422672,
 'Spironolactone Oral': 0.1796726581052179,
 'Aspirin Chewable Tab': 0.17775945434183948,
 'Ferric Hydr Sucrose Inj': 0.17736057145883305,
 'Isosorbide Dinitrate Tab': 0.17723192703738708,
 'Enoxaparin Inj': 0.17455386056663816,
 'Insulin Reg Inj (HumuLIN R)': 0.17383471882637963,
 'Sodium Polystyrene Sulfonate Oral': 0.17353292244336818,
 'Carvedilol Tab': 0.17350246662273835,
 'DOPamine DRIP': 0.17342011312254063,
 'Atorvastatin Tab': 0.1728722774

In [202]:
N

96